In [1]:
%pip install yfinance
%pip install pandas
%pip install numpy
%pip install matplotlib
%pip install statsmodels
%pip install datetime
%pip install scikit-learn
%pip install PyPortfolioOpt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 8.3 MB/s eta 0:00:00


In [2]:
%pip install pandas-ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218910 sha256=f2d9b3ad49f1dc83db626697b9458d71c5612b25f3ebb03083348bc881f5be5e
  Stored in directory: /root/.cache/pip/wheels/7f/33/8b/50b245c5c65433cd8f5cb24ac15d97e5a3db2d41a8b6ae957d
Successfully built pandas-ta


## Load SP500 Data

In [6]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas_ta
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import warnings

In [7]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
symbols_list = sp500['Symbol'].unique().tolist()

In [8]:
end_date = '2025-01-01'

start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8)

In [9]:
df = yf.download(tickers=symbols_list, start=start_date, end=end_date, auto_adjust=False)
df.stack()

[*********************100%***********************]  503 of 503 completed
<ipython-input-9-387965520>:2: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.stack()


Price               Adj Close       Close        High         Low        Open  \
Date       Ticker                                                               
2017-01-03 A        43.651127   46.490002   46.750000   45.740002   45.930000   
           AAPL     26.827240   29.037500   29.082500   28.690001   28.950001   
           ABBV     43.450211   62.410000   63.029999   61.939999   62.919998   
           ABT      33.456707   39.049999   39.080002   38.340000   38.630001   
           ACGL     27.224224   28.629999   29.143333   28.506666   28.943333   
...                       ...         ...         ...         ...         ...   
2024-12-31 XYL     115.293098  116.019997  117.110001  115.570000  116.779999   
           YUM     132.877213  134.160004  134.789993  133.250000  134.089996   
           ZBH     105.404167  105.629997  106.500000  104.959999  105.910004   
           ZBRA    386.220001  386.220001  387.410004  381.750000  383.420013   
           ZTS     161.892609  162.929993  163.750000  161.869995  162.899994   

Price                   Volume  
Date       Ticker               
2017-01-03 A         1739600.0  
           AAPL    115127600.0  
           ABBV      9328200.0  
           ABT       9677300.0  
           ACGL       942900.0  
...                        ...  
2024-12-31 XYL        641600.0  
           YUM       1217100.0  
           ZBH        683300.0  
           ZBRA       327900.0  
           ZTS       1327400.0  

[989160 rows x 6 columns]

In [10]:
df = df.stack()

<ipython-input-10-3911042461>:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df = df.stack()


## Calculate features and technical indicators for each stock

In [13]:
from typing_extensions import dataclass_transform
# Garman-Klass Volatility
df['garman_klass_vol']=((np.log(df['High']) - np.log(df['Low']))**2)/2 - (2*np.log(2)-1)*((np.log(df['Adj Close']) - np.log(df['Open']))**2)

# RSI
df['rsi'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

# BBL
df['bb_low'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])
df['bb_mid'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
df['bb_high'] = df.groupby(level=1)['Adj Close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

In [14]:
# ATR
def compute_atr(stock_data):
  atr = pandas_ta.atr(high=stock_data['High'], low=stock_data['Low'], close=stock_data['Close'], length=14)

  return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

In [16]:
# MACD
def compute_macd(close):
  macd = pandas_ta.macd(close=close, length=20).iloc[:,0]

  return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['Adj Close'].apply(compute_macd)

In [17]:
# Dollar Volume
df['dollar_volume'] = (df['Adj Close']*df['Volume'])/1e6

## Aggregate to montly level and filter top 150 most liquid stocks for each month